<a href="https://colab.research.google.com/github/andreianmatos/temporal_spaces_texture_gen/blob/main/styleGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Use of an open-source PyTorch implementation of StyleGAN2 - https://github.com/lucidrains/stylegan2-pytorch - setup for training using Colab's free GPU resources and Google Drive - https://github.com/96jonesa/StyleGan2-Colab-Demo .

Training results and models are saved to the local runtime's 'results' and 'models' directories (folder icon on left bar), or to your Google Drive (give access in cell  below) in subdirectories of a parent directory named 'StyleGan2_small_set_demo'.


# StyleGan2:

<div>
<img src="https://github.com/andreianmatos/temporal_spaces_texture_gen/blob/main/stylegan_gif.gif?raw=true" width="400"/>
</div>

0. Login to Google (Drive)
1. Click 'Copy to Drive' above to make a runnable copy of this notebook.
2. Run this cell (click the play button in top left of cell) to connect to a runtime instance.
3. Navigate to 'Runtime > Change Runtime Type > Hardware Accelerator' and select GPU.
4. If needed, modify the variables found in the cell below to select behavior of demo.
5. Run cells.

In [6]:
USE_DATASET = 'all_members_64'

TRAINING_FROM_SCRATCH = True # set True if training from scratch, False if training from last checkpoint
MODEL_NAME = 'model'
MODEL_NUM_TRAIN_STEPS = 3000

LOW_NETWORK_CAPACITY = False # set True to use significantly lower network capacity
USE_GOOGLE_DRIVE_FOR_TRAINING = True # save models and results directly to your Google Drive

# 'none', 'first', 'every'
USE_ATTENTION_LAYERS = 'none' # which layers do you want attention applied to?
# dataset consists of relatively simple patterns and structures, and you don't observe significant long-range dependencies in your images

MODEL_AUGMENTATION_PROBABILITY = 0
MODEL_LEARNING_RATE = 2e-4
MODEL_IMAGE_SIZE = 64

In [7]:
# Mounts your Google Drive so files can be saved to it. Note that this also allows
# files to be read from it, so only authorize this if you are comfortable doing so
# and/or using a disposable Google Drive account.

if USE_GOOGLE_DRIVE_FOR_TRAINING:
    from google.colab import drive
    drive.mount('/content/drive')

    !mkdir -p "/content/drive/My Drive/StyleGan2_small_set_demo"

Mounted at /content/drive


In [8]:
MODEL_NETWORK_CAPACITY = 16
if LOW_NETWORK_CAPACITY:
    MODEL_NETWORK_CAPACITY = 4

MODEL_ATTENTION_LAYERS = []
if USE_ATTENTION_LAYERS == 'first':
    MODEL_ATTENTION_LAYERS = "[1]"
elif USE_ATTENTION_LAYERS == 'every':
    MODEL_ATTENTION_LAYERS = "[1,2,3,4,5,6]"

MODEL_NAME = USE_DATASET + '_' + MODEL_NAME

In [9]:
# Installs the architecture from:
# https://github.com/lucidrains/stylegan2-pytorch

!pip install stylegan2_pytorch==0.17.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for adamp: filename=adamp-0.3.0-py3-none-any.whl size=5983 sha256=e11c9fd06ab547010e289baefce7b88f23b9080517c1e0c9ccd332d670a9fb84
  Stored in directory: /root/.cache/pip/wheels/c7/ad/0f/b41b1c45b18c66e5eef5d2254415af8055c7e2b0934145157d
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=90d13e56865ca5929041a7540999a7a3617ca3d5fdd2793dbd32e92cc83196aa
  Stored in directory: /root

In [10]:
# Utilities for downloading publicly shared Google Drive files (from your Google Drive).

import requests

def download_file_from_google_drive(id, destination):
    URL = 'https://docs.google.com/uc?export=download'

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, 'wb') as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [11]:
!pip install linformer

In [12]:
# Downloads and unzips the selected dataset from your Google Drive.

import zipfile

if USE_DATASET == 'all_members_64':

    file_id = '1EjMb16e_tJ4YT-46WbSURKgB4pxureMC'
    destination = 'all_members_64.zip'
    download_file_from_google_drive(file_id, destination)
    zip_ref = zipfile.ZipFile('all_members_64.zip', 'r')
    zip_ref.extractall('data/all_members_64')
    zip_ref.close()

In [13]:
# Chooses the appropriate subdirectory of dataset for training.
MODEL_DATA_DIR = 'data/all_members_64/'

In [14]:
# Establish directories for custom models.

CUSTOM_RESULTS_DIR = './results'
CUSTOM_MODELS_DIR = './models'

if USE_GOOGLE_DRIVE_FOR_TRAINING:
    CUSTOM_RESULTS_DIR = '"/content/drive/My Drive/StyleGan2_small_set_demo/results"'
    CUSTOM_MODELS_DIR = '"/content/drive/My Drive/StyleGan2_small_set_demo/models"'

In [15]:

# Train custom models.

if TRAINING_FROM_SCRATCH:
    !stylegan2_pytorch --data {MODEL_DATA_DIR} --name {MODEL_NAME} --new --network_capacity {MODEL_NETWORK_CAPACITY} --batch_size 4 \
        --gradient_accumulate_every 4 --num_train_steps {MODEL_NUM_TRAIN_STEPS} --attn_layers {MODEL_ATTENTION_LAYERS} --image_size {MODEL_IMAGE_SIZE} \
        --aug_prob {MODEL_AUGMENTATION_PROBABILITY} --results_dir {CUSTOM_RESULTS_DIR} --models_dir {CUSTOM_MODELS_DIR} --learning_rate {MODEL_LEARNING_RATE} --num_image_tiles 2
else:
    !stylegan2_pytorch --data {MODEL_DATA_DIR} --name {MODEL_NAME} --network_capacity {MODEL_NETWORK_CAPACITY} --batch_size 4 \
        --gradient_accumulate_every 4 --num_train_steps {MODEL_NUM_TRAIN_STEPS} --attn_layers {MODEL_ATTENTION_LAYERS} --image_size {MODEL_IMAGE_SIZE} \
        --aug_prob {MODEL_AUGMENTATION_PROBABILITY} --results_dir {CUSTOM_RESULTS_DIR} --models_dir {CUSTOM_MODELS_DIR} --learning_rate {MODEL_LEARNING_RATE}

all_members_64_model<data/all_members_64/>:   0% 0/3000 [00:00<?, ?it/s]G: 66.76 | D: 0.95 | GP: 164107.88 | PL: 0.01 | CR: 0.00 | Q: 0.00
all_members_64_model<data/all_members_64/>:   1% 34/3000 [00:22<28:05,  1.76it/s]  G: 27.77 | D: 13.34 | GP: 11760.93 | PL: 0.02 | CR: 0.00 | Q: 0.00
all_members_64_model<data/all_members_64/>:   3% 97/3000 [00:43<18:39,  2.59it/s]G: 9.07 | D: 11.65 | GP: 3155.72 | PL: 0.02 | CR: 0.00 | Q: 0.00
all_members_64_model<data/all_members_64/>:   4% 128/3000 [00:53<17:29,  2.74it/s]G: 3.02 | D: 7.61 | GP: 839.01 | PL: 0.02 | CR: 0.00 | Q: 0.00
all_members_64_model<data/all_members_64/>:   6% 192/3000 [01:14<16:02,  2.92it/s]G: 5.38 | D: 3.63 | GP: 273.19 | PL: 0.03 | CR: 0.00 | Q: 0.00
all_members_64_model<data/all_members_64/>:   7% 222/3000 [01:25<16:01,  2.89it/s]G: 1.31 | D: 1.87 | GP: 119.04 | PL: 0.03 | CR: 0.00 | Q: 0.00
all_members_64_model<data/all_members_64/>:  10% 288/3000 [01:46<14:56,  3.02it/s]G: 0.70 | D: 1.50 | GP: 31.01 | PL: 0.03 | CR: 0

In [16]:
!stylegan2_pytorch --generate_interpolation --name {MODEL_NAME} --num_image_tiles 1  --image_size {MODEL_IMAGE_SIZE} --results_dir {CUSTOM_RESULTS_DIR + "/generated_interpolation"} --models_dir {CUSTOM_MODELS_DIR}

continuing from previous epoch - 2
100% 100/100 [00:01<00:00, 51.98it/s]
interpolation generated at /content/drive/My Drive/StyleGan2_small_set_demo/results/generated_interpolation/all_members_64_model/generated-01-23-2024_11-28-39


In [17]:
number_images_to_generate = 10
for el in range(number_images_to_generate):
  !stylegan2_pytorch --generate --name {MODEL_NAME} --num_image_tiles 1  --image_size {MODEL_IMAGE_SIZE} --results_dir {CUSTOM_RESULTS_DIR  + "/generated_images"} --models_dir {CUSTOM_MODELS_DIR}

continuing from previous epoch - 2
sample images generated at /content/drive/My Drive/StyleGan2_small_set_demo/results/generated_images/all_members_64_model/generated-01-23-2024_11-28-47
continuing from previous epoch - 2
sample images generated at /content/drive/My Drive/StyleGan2_small_set_demo/results/generated_images/all_members_64_model/generated-01-23-2024_11-28-54
continuing from previous epoch - 2
sample images generated at /content/drive/My Drive/StyleGan2_small_set_demo/results/generated_images/all_members_64_model/generated-01-23-2024_11-28-59
continuing from previous epoch - 2
sample images generated at /content/drive/My Drive/StyleGan2_small_set_demo/results/generated_images/all_members_64_model/generated-01-23-2024_11-29-06
continuing from previous epoch - 2
sample images generated at /content/drive/My Drive/StyleGan2_small_set_demo/results/generated_images/all_members_64_model/generated-01-23-2024_11-29-11
continuing from previous epoch - 2
sample images generated at /co

## Parameters accepted by model:

In [5]:
# parameter                 | default   | description
#                           |           |
# data                      | ./data    | directory containing data
# results_dir               | ./results | directory for checkpoint sample images
# models_dir                | ./models  | directory for checkpoint models (saves to and loads from here)
# name                      | default   | name to identify model (all outputs will be saved to results_dir/name and models_dir/name)
# new                       | False     | if True then starts from scratch, else loads from saved checkpoint model
# load_from                 | -1        | if -1 then loads from most recent checkpoint, else loads from checkpoint number load_from
# image_size                | 128       | size of (square) images generated and for resizing of data
# network_capacity          | 16        | affects number of nodes per layer - decrease to train faster with lower output quality
# transparent               | False     | if True then uses RGBA, else uses RGB
# batch_size                | 3         | number of images per mini-batch (larger uses more GPU memory)
# gradient_accumulate_every | 5         | number of mini-batches to process before optimizing (choice depends on batch_size)
# num_train_steps           | 150000    | total steps of forward prop (counting starts from number of steps completed in loaded checkpoint)
# learning_rate             | 2e-4      | learning rate
# num_workers               | None      | if None then uses as many workers as possible from available CPU cores (for data loading)
# save_every                | 1000      | every save_every steps, a checkpoint model and sample images are saved
# generate                  | False     | if True then generates sample images from loaded model instead of training
# generate_interpolation    | False     | if True then generates .gif interpolation from loaded model instead of training, else does not
# num_image_tiles           | 8         | generated samples will be a grid of (num_image_tiles x num_image_tiles) images
# trunc_psi                 | 0.75      | affects how far generate images can be from average image (increase for more diversity) w_new = psi * w + (1 - psi) * w_avg
# fp16                      | False     | if True then uses fp16 half-precision to lower GPU memory usage (requires apex), else uses full-precision
# cl_reg                    | False     | if True then uses contrastive learning on discriminator (possibly improves stability and quality), else does not
# fq_layers                 | []        | list of layers to apply feature (intermediate representation) vector quantization to (can improve results, but not dramatically)
# fq_dict_size              | 256       | dictionary size for feature quantization
# attn_layers               | []        | list of layers to apply self-attention to while training (can be empty; do not use spaces; up to log2(image_size) - 1 layers)
# no_const                  | False     | if True then 4x4 block is learned from style vector, else styles a constant learned 4x4 block through progressive upsampling
# aug_prob                  | 0.0       | probability of applying differentiable augmentation to images fed to discriminator